In [1]:
# %pip install openai==1.63.2

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 5, Finished, Available, Finished)

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import datetime
from numpy import vectorize
import openai
from openai import OpenAI
import pandas as pd
import random
import os
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

api_key=mssparkutils.credentials.getSecret(
    "https://ventagiumcommonkeyvault.vault.azure.net/",
    "Open-AI-Token"
)


client = OpenAI(
    api_key=api_key
)

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 33, Finished, Available, Finished)

## Data Preparation

In [30]:
import pandas as pd
from datetime import datetime

# Data ofr reviews
reviews_data = [
    (datetime(2024, 6, 30), 1, "Excellent sound quality and battery life!", 5),
    (datetime(2024, 2, 29), 1, "Connection issues at times.", 2),
    (datetime(2024, 8, 12), 1, "Decent, but expected more for the price.", 3),
    (datetime(2024, 12, 10), 1, "Terrible sound and hurts my ears!", 1),
    (datetime(2024, 4, 28), 1, "Best headphones I've ever had!", 5),
    (datetime(2024, 9, 11), 1, "Amazing noise cancellation!", 5),
    (datetime(2024, 4, 9), 1, "Decent, but expected more for the price.", 3),
    (datetime(2024, 12, 28), 1, "Sound is fine, but build quality is average.", 3),
    (datetime(2024, 5, 27), 1, "Good for travel, but ear cups feel cheap.", 3),
    (datetime(2024, 4, 11), 1, "Battery lasts all day, impressive!", 5),
    (datetime(2024, 10, 5), 2, "Amazing features and great battery life!", 5),
    (datetime(2024, 6, 16), 2, "Good watch, but wish it had better integration.", 4),
    (datetime(2024, 3, 20), 2, "Average watch, nothing special.", 3),
    (datetime(2024, 1, 11), 2, "Super responsive and stylish.", 5),
    (datetime(2024, 4, 20), 2, "Good, but screen scratches too easily.", 3),
    (datetime(2024, 1, 7), 2, "Battery drains too fast, disappointing.", 2),
    (datetime(2024, 8, 20), 2, "Limited apps available, but nice build.", 3),
    (datetime(2024, 7, 12), 2, "Not worth the price, very buggy.", 2),
    (datetime(2024, 12, 25), 3, "Compact and good performance.", 4),
    (datetime(2024, 6, 30), 3, "Decent bass but lacks clarity.", 3),
    (datetime(2024, 7, 9), 3, "Loud and clear, perfect for parties!", 5),
    (datetime(2024, 11, 14), 3, "Not bad, but could be louder.", 3),
    (datetime(2024, 5, 7), 3, "Compact and good performance.", 4),
    (datetime(2024, 2, 12), 3, "Broke within a week.", 1),
    (datetime(2024, 11, 23), 3, "Solid build and decent sound quality.", 4),
    (datetime(2024, 1, 17), 3, "Good for small rooms, not for outdoors.", 3),
    (datetime(2024, 4, 9), 3, "Disappointed with the Bluetooth range.", 2),
    (datetime(2024, 3, 31), 3, "Excellent for its size and price.", 5),
    (datetime(2024, 12, 31), 4, "Works well, but instructions could be clearer.", 4),
    (datetime(2024, 11, 16), 4, "Great results but takes up space.", 4),
    (datetime(2024, 7, 17), 4, "Cooks evenly and quickly!", 5),
    (datetime(2024, 3, 12), 4, "Healthy and delicious food every time!", 5),
    (datetime(2024, 11, 10), 4, "Not as powerful as I expected.", 3),
    (datetime(2024, 1, 7), 4, "Takes a bit to clean, but works well.", 4),
    (datetime(2024, 3, 6), 4, "Cooks perfectly and super easy to clean!", 5),
    (datetime(2024, 4, 26), 4, "Could use a better timer function.", 3),
    (datetime(2024, 4, 29), 5, "Weak suction, completely useless!", 1),
    (datetime(2024, 9, 21), 5, "Makes cleaning effortless!", 5),
    (datetime(2024, 12, 5), 5, "Super powerful suction, love it!", 5),
    (datetime(2024, 1, 3), 5, "Does the job, but nothing amazing.", 3),
    (datetime(2024, 3, 31), 5, "Gets clogged too easily.", 2),
    (datetime(2024, 9, 8), 5, "Works well, but a bit noisy.", 4),
    (datetime(2024, 4, 25), 5, "Average suction power.", 3),
    (datetime(2024, 2, 3), 5, "Good for the price, but battery could be better.", 4),
    (datetime(2024, 3, 14), 5, "Decent but struggles with carpets.", 3),
    (datetime(2024, 11, 7), 5, "Fantastic for pet hair, highly recommended!", 5),
    (datetime(2024, 4, 19), 5, "Gets stuck sometimes, needs improvement.", 2)
]


# Create DataFrame for reviews
df_reviews_pandas = pd.DataFrame(reviews_data, columns=["review_date", "product_id", "review", "stars"])

# Data for products
products_data = [
    (1, "Electronics", "Wireless Headphones"),
    (2, "Electronics", "Smart Watch"),
    (3, "Electronics", "Bluetooth Speaker"),
    (4, "Home Appliances", "Air Fryer"),
    (5, "Home Appliances", "Robot Vacuum")
]

# Crear DataFrame for products
df_products_pandas = pd.DataFrame(products_data, columns=["product_id", "product_category", "product_name"])

# Display DataFrames
df_reviews_pandas.head(), df_products_pandas.head()


# Define suppliers per product
suppliers = {
    1: ("AudioTech", "SoundWave"),
    2: ("SmartWear", "TechTime"),
    3: ("BassBoost", "ClearSound"),
    4: ("HomeChef", "KitchenPro"),
    5: ("CleanMaster", "RoboClean")
}

# Define shipping companies
shipping_companies = ["FastShip", "ReliableExpress", "EcoDelivery"]

def assign_shipping(review_date, stars):
    """Assigns a shipping company based on the date and sentiment pattern."""
    if stars >= 4:
        return shipping_companies[review_date.month % 3]  # Cycle through months
    elif stars == 3:
        return shipping_companies[(review_date.month + 1) % 3]
    else:
        return shipping_companies[(review_date.month + 2) % 3]

# Extend reviews data with suppliers and shipping companies
df_reviews_pandas["supplier"] = df_reviews_pandas["product_id"].apply(lambda x: random.choice(suppliers[x]))
df_reviews_pandas["shipping_company"] = df_reviews_pandas.apply(lambda row: assign_shipping(row["review_date"], row["stars"]), axis=1)

# Display modified DataFrame
df_reviews_pandas.head(), df_products_pandas.head()


StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 34, Finished, Available, Finished)

(  review_date  product_id                                     review  stars  \
 0  2024-06-30           1  Excellent sound quality and battery life!      5   
 1  2024-02-29           1                Connection issues at times.      2   
 2  2024-08-12           1   Decent, but expected more for the price.      3   
 3  2024-12-10           1          Terrible sound and hurts my ears!      1   
 4  2024-04-28           1             Best headphones I've ever had!      5   
 
     supplier shipping_company  
 0  AudioTech         FastShip  
 1  SoundWave  ReliableExpress  
 2  SoundWave         FastShip  
 3  SoundWave      EcoDelivery  
 4  SoundWave  ReliableExpress  ,
    product_id product_category         product_name
 0           1      Electronics  Wireless Headphones
 1           2      Electronics          Smart Watch
 2           3      Electronics    Bluetooth Speaker
 3           4  Home Appliances            Air Fryer
 4           5  Home Appliances         Robot Vacuum)

## Sentiment Analysis

In [31]:
@vectorize
def analyze_sentiment(text):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a sentiment analysis classifier."},
                  {"role": "user", "content": f"Classify the sentiment of this text as Positive, Negative, or Neutral: {text}"}]
    )
    return response.choices[0].message.content

df_reviews_pandas['sentiment'] = analyze_sentiment(df_reviews_pandas['review'])

df_reviews_pandas.head()

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 35, Finished, Available, Finished)

,review_date,product_id,review,stars,supplier,shipping_company,sentiment
0,2024-06-30,1,Excellent sound quality and battery life!,5,AudioTech,FastShip,Positive
1,2024-02-29,1,Connection issues at times.,2,SoundWave,ReliableExpress,Negative
2,2024-08-12,1,"Decent, but expected more for the price.",3,SoundWave,FastShip,Negative
3,2024-12-10,1,Terrible sound and hurts my ears!,1,SoundWave,EcoDelivery,Negative
4,2024-04-28,1,Best headphones I've ever had!,5,SoundWave,ReliableExpress,Positive


##  Generating Review Summaries

In [32]:
# Step 1: Summarize client reviews for each product using OpenAI
def summarize_reviews(product_name, reviews):
    prompt = f"Please summarize the following customer reviews for the product '{product_name}':\n\n" + "\n".join(reviews)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or any suitable model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Step 2: Analysis by product
def generate_product_insights(df_reviews, df_products):
    insights = []
    monthly_avg_ratings = []

    # Iterate over each product
    for _, product in df_products.iterrows():
        product_name = product['product_name']
        product_id = product['product_id']

        # Filter reviews for this product
        product_reviews = df_reviews[df_reviews['product_id'] == product_id]

        # Summarize reviews using OpenAI
        reviews_text = product_reviews['review'].tolist()
        summary = summarize_reviews(product_name, reviews_text)
        
        # Calculate average rating
        avg_rating = product_reviews['stars'].mean()

        # Trends over time (e.g., average rating by month)
        product_reviews['review_month'] = product_reviews['review_date'].dt.to_period('M')
        monthly_avg_rating = product_reviews.groupby('review_month')['stars'].mean().reset_index()

        # Store insights for the product
        insights.append({
            "product_name": product_name,
            "summary": summary,
            "average_rating": avg_rating
        })
        
        # Append the monthly ratings for further analysis
        monthly_avg_ratings.append(monthly_avg_rating.assign(product_name=product_name))

    # Convert insights to a DataFrame
    insights_df = pd.DataFrame(insights)

    # Combine monthly average ratings from all products
    monthly_avg_ratings_df = pd.concat(monthly_avg_ratings, ignore_index=True)

    return insights_df, monthly_avg_ratings_df

# Get insights and store them in DataFrames
insights_df, monthly_avg_ratings_df = generate_product_insights(df_reviews_pandas, df_products_pandas)

# Display the insights DataFrame
print(insights_df)

# Display the monthly average ratings DataFrame
print(monthly_avg_ratings_df)

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 36, Finished, Available, Finished)

/tmp/ipykernel_16681/986527203.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_reviews['review_month'] = product_reviews['review_date'].dt.to_period('M')


In [33]:
# Function to analyze supplier issues
def analyze_supplier_issues(supplier, reviews):
    prompt = f"Summarize the main issues for supplier '{supplier}' based on the following reviews:\n\n" + "\n".join(reviews)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Function to analyze shipping company reviews
def analyze_shipping_reviews(company, reviews):
    prompt = f"Provide insights on customer satisfaction for shipping company '{company}' based on these reviews:\n\n" + "\n".join(reviews)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Analyze failures per supplier
supplier_analysis = df_reviews_pandas.groupby("supplier")["stars"].agg(["count", "mean"]).reset_index()
supplier_analysis.rename(columns={"count": "review_count", "mean": "average_stars"}, inplace=True)

# Generate insights for suppliers
supplier_analysis["issues_summary"] = supplier_analysis["supplier"].apply(
    lambda supplier: analyze_supplier_issues(supplier, df_reviews_pandas[df_reviews_pandas["supplier"] == supplier]["review"].tolist())
)

# Analyze review and rating insights per shipping company
shipping_analysis = df_reviews_pandas.groupby("shipping_company")["stars"].agg(["count", "mean"]).reset_index()
shipping_analysis.rename(columns={"count": "review_count", "mean": "average_stars"}, inplace=True)

# Generate insights for shipping companies
shipping_analysis["review_insights"] = shipping_analysis["shipping_company"].apply(
    lambda company: analyze_shipping_reviews(company, df_reviews_pandas[df_reviews_pandas["shipping_company"] == company]["review"].tolist())
)

# Display results
print("Supplier Analysis:")
print(supplier_analysis)
print("\nShipping Company Analysis:")
print(shipping_analysis)

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 37, Finished, Available, Finished)

Supplier Analysis:
      supplier  review_count  average_stars  \
0    AudioTech             3       3.666667   
1    BassBoost             2       4.500000   
2  CleanMaster             5       3.200000   
3   ClearSound             8       3.125000   
4     HomeChef             4       4.500000   
5   KitchenPro             4       3.750000   
6    RoboClean             6       3.500000   
7    SmartWear             5       3.600000   
8    SoundWave             7       3.428571   
9     TechTime             3       3.000000   

                                      issues_summary  
0  The main issues for supplier 'AudioTech' based...  
1  It seems that the main issues for supplier 'Ba...  
2  The main issues for supplier CleanMaster based...  
3  The main issues for supplier 'ClearSound' base...  
4  The main issues for the supplier 'HomeChef' ba...  
5  Based on the reviews, the main issues for supp...  
6  The main issues for supplier 'RoboClean' based...  
7  Based on the reviews

In [34]:
supplier_analysis.head()

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 38, Finished, Available, Finished)

,supplier,review_count,average_stars,issues_summary
0,AudioTech,3,3.666667,The main issues for supplier 'AudioTech' based...
1,BassBoost,2,4.500000,It seems that the main issues for supplier 'Ba...
2,CleanMaster,5,3.200000,The main issues for supplier CleanMaster based...
3,ClearSound,8,3.125000,The main issues for supplier 'ClearSound' base...
4,HomeChef,4,4.500000,The main issues for the supplier 'HomeChef' ba...


In [35]:
monthly_avg_ratings_df['review_month'] = pd.to_datetime(monthly_avg_ratings_df['review_month'].dt.to_timestamp())

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 39, Finished, Available, Finished)

## Loading Files to Lakehouse

In [41]:
# Convert Pandas DataFrame to Spark DataFrame
insights_spark_df = spark.createDataFrame(insights_df)
monthly_avg_ratings_spark_df = spark.createDataFrame(monthly_avg_ratings_df)
df_reviews_spark = spark.createDataFrame(df_reviews_pandas)
supplier_analysis_spark = spark.createDataFrame(supplier_analysis)
shipping_analysis_spark = spark.createDataFrame(shipping_analysis)

# Save insights_df as a Delta table
insights_spark_df.write.format("delta").mode("overwrite").save("abfss://Fabric_Workshop@onelake.dfs.fabric.microsoft.com/Fabric_Workshop_Lakehouse.Lakehouse/Tables/OpenAI_Reviews_Insights")

# Save monthly_avg_ratings_df as a Delta table
monthly_avg_ratings_spark_df.write.format("delta").mode("overwrite").save("abfss://Fabric_Workshop@onelake.dfs.fabric.microsoft.com/Fabric_Workshop_Lakehouse.Lakehouse/Tables/OpenAI_Monthly_Ratings")

# Save df_reviews_pandas as a Delta table
df_reviews_spark.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("abfss://Fabric_Workshop@onelake.dfs.fabric.microsoft.com/Fabric_Workshop_Lakehouse.Lakehouse/Tables/OpenAI_Reviews")

# Save supplier_analysis as a Delta table
supplier_analysis_spark.write.format("delta").mode("overwrite").save("abfss://Fabric_Workshop@onelake.dfs.fabric.microsoft.com/Fabric_Workshop_Lakehouse.Lakehouse/Tables/OpenAI_Supplier_Analysis")

# Save shipping_analysis as a Delta table
shipping_analysis_spark.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("abfss://Fabric_Workshop@onelake.dfs.fabric.microsoft.com/Fabric_Workshop_Lakehouse.Lakehouse/Tables/OpenAI_Shipping_Analysis")

StatementMeta(, 40ddc59e-62c2-4a13-9186-8d73840a9c17, 45, Finished, Available, Finished)